In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.callbacks import LambdaCallback, ModelCheckpoint
import numpy as np
import random
import sys
import io
import os

In [ ]:
def load_data(file):
    data = open(file, 'r', encoding='utf-8')
    text = data.read()
    data.close()
    return text
    
train_data = "kanye.txt"

text = load_data(train_data)

chars = sorted(list(set(text)))

char_ind = dict((c, i) for i, c in enumerate(chars))
ind_char = dict((i, c) for i, c in enumerate(chars))

print(char_ind)

In [4]:
def sample(preds, diversity=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / diversity
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2, 1.5]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_ind[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = ind_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        
generate_lyrics = LambdaCallback(on_epoch_end=on_epoch_end)

In [5]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_ind[char]] = 1
    y[i, char_ind[next_chars[i]]] = 1

In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.2))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [12]:
filepath = "lyrics-weights.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             monitor='loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='min')

In [13]:
 model.fit(x, y,
              batch_size=128,
              epochs=25,
              verbose=2,
              callbacks=[generate_lyrics, checkpoint])

Epoch 1/25

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "plea that I'm copping
I'm just willing t"
plea that I'm copping
I'm just willing to see the fuck it and the fuck a back to the start to be the way to see the couch a fuck it the this face to the back and the whought the couch the couse the fuck the don't be the back and sour the baby with the back to see the sour the back to see the back and sore shit the fuck the bout the back the short your sour the back the strick of the back the fuck a back to the thing of the back to be th
----- diversity: 0.5
----- Generating with seed: "plea that I'm copping
I'm just willing t"
plea that I'm copping
I'm just willing to strick it till got start it the diggin here from the boys for me (I all the way me wores in the resure me we go that some more just feel the sure me I want for the whole me to crack going to said your sours to the day the way this the free I can't know you just want of your sick coul

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


ke I was do to stay the pain the mind and get on the painstappors on the time and I'm a blood and I don't
----- diversity: 1.0
----- Generating with seed: "pper lose custodyExalt, Exalt, glorifyDe"
pper lose custodyExalt, Exalt, glorifyDey ,44S5,"]
show geeps
Squeer it

Just doubt homin' surd...""
Put these ghar a penticsays Goodb that
Gave weet Here give these yead, it forget clut feeled to everybody three conds, so shit
cleanness to g"t! Well your under mample, that's some sing-ana it?
If you ain't make the first in here fan, bitch how take oness flaps im canor
I'm mixesthy, this is the dauth, rap I told his selfotent up drink, 
----- diversity: 1.2
----- Generating with seed: "pper lose custodyExalt, Exalt, glorifyDe"
pper lose custodyExalt, Exalt, glorifyDearth,3633,

Ove is squee, so a I pefsingSy's and rapple's death beef augrt I know we rapping up talkin out!
Or the Rund shake tap by Hanrer)
Heym soxim Loon)
But no Raw's taok at than I'll penceble, you own the day it is I need a

True